In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from time import time
pd.set_option('display.max_columns', 999)
from sklearn.metrics import accuracy_score
from mlxtend.evaluate import lift_score

In [3]:
traindf = pd.read_csv('/Users/Vishy/Files/AVDatahack/CrossSell/RawData/train.csv')
#traindf = pd.read_csv('/data/train.csv')
print (traindf.shape)
traindf.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(300000, 333)


,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,EEG_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,EEG_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,PM_FD_MON_02,PM_FD_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,MATURITY_GL,MATURITY_LAP,MATURITY_LAS,STMT_CON_DAE_ACTIVE_MON_01,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,STMT_CON_DAE_CLOSED_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,MER_EMI_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_

In [4]:
# There are 333 variables in the training set data
nonnumcols = traindf.dtypes[traindf.dtypes == "object"].index
print (len(nonnumcols))
#print (nonnumcols)

93


In [4]:
# Only Y columns
traindf['BL_TAG'] = np.where(traindf['BL_TAG']=='Y', 1, 0)
traindf['TWL_TAG'] = np.where(traindf['TWL_TAG']=='Y', 1, 0)
traindf['PL_TAG'] = np.where(traindf['PL_TAG']=='Y', 1, 0)
traindf['AL_CNC_TAG'] = np.where(traindf['AL_CNC_TAG']=='Y', 1, 0)
traindf['AGRI_TAG'] = np.where(traindf['AGRI_TAG']=='Y', 1, 0)
traindf['CE_TAG'] = np.where(traindf['CE_TAG']=='Y', 1, 0)
traindf['CV_TAG'] = np.where(traindf['CV_TAG']=='Y', 1, 0)
traindf['EDU_TAG'] = np.where(traindf['EDU_TAG']=='Y', 1, 0)
traindf['TL_TAG'] = np.where(traindf['TL_TAG']=='Y', 1, 0)
traindf['OTHER_LOANS_TAG'] = np.where(traindf['OTHER_LOANS_TAG']=='Y', 1, 0)
traindf['LAP_TAG'] = np.where(traindf['LAP_TAG']=='Y', 1, 0)
traindf['LAS_TAG'] = np.where(traindf['LAS_TAG']=='Y', 1, 0)
traindf['BBG_TAG'] = np.where(traindf['BBG_TAG']=='Y', 1, 0)
traindf['HL_TAG'] = np.where(traindf['HL_TAG']=='Y', 1, 0)
traindf['GL_TAG'] = np.where(traindf['GL_TAG']=='Y', 1, 0)
traindf['AL_CLOSED'] = np.where(traindf['AL_CLOSED']=='Y', 1, 0)
traindf['PL_CLOSED'] = np.where(traindf['PL_CLOSED']=='Y', 1, 0)
traindf['BL_CLOSED'] = np.where(traindf['BL_CLOSED']=='Y', 1, 0)

In [5]:
# Only y's 2
traindf['TWL_CLOSED'] = np.where(traindf['TWL_CLOSED']=='Y', 1, 0)
traindf['CV_CLOSED'] = np.where(traindf['CV_CLOSED']=='Y', 1, 0)
traindf['LAP_CLOSED'] = np.where(traindf['LAP_CLOSED']=='Y', 1, 0)
traindf['LAS_CLOSED'] = np.where(traindf['LAS_CLOSED']=='Y', 1, 0)
traindf['BBG_CLOSED'] = np.where(traindf['BBG_CLOSED']=='Y', 1, 0)
traindf['GL_CLOSED'] = np.where(traindf['GL_CLOSED']=='Y', 1, 0)
traindf['CC_HOLD_MON_03'] = np.where(traindf['CC_HOLD_MON_03']=='Y', 1, 0)
traindf['CC_HOLD_MON_02'] = np.where(traindf['CC_HOLD_MON_02']=='Y', 1, 0)
traindf['CC_HOLD_MON_01'] = np.where(traindf['CC_HOLD_MON_01']=='Y', 1, 0)
traindf['CC_ACTIVE_MON_03'] = np.where(traindf['CC_ACTIVE_MON_03']=='Y', 1, 0)
traindf['CC_ACTIVE_MON_02'] = np.where(traindf['CC_ACTIVE_MON_02']=='Y', 1, 0)
traindf['CC_ACTIVE_MON_01'] = np.where(traindf['CC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['DC_ACTIVE_MON_03'] = np.where(traindf['DC_ACTIVE_MON_03']=='Y', 1, 0)
traindf['DC_ACTIVE_MON_02'] = np.where(traindf['DC_ACTIVE_MON_02']=='Y', 1, 0)
traindf['DC_ACTIVE_MON_01'] = np.where(traindf['DC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['BTE_ACTIVE_MON_01'] = np.where(traindf['BTE_ACTIVE_MON_01']=='Y', 1, 0)
traindf['COC_ACTIVE_MON_01'] = np.where(traindf['COC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['PL2CC_ACTIVE_MON_01'] = np.where(traindf['PL2CC_ACTIVE_MON_01']=='Y', 1, 0)
traindf['MER_EMI_ACTIVE_MON_01'] = np.where(traindf['MER_EMI_ACTIVE_MON_01']=='Y', 1, 0)

In [6]:
# Only y's 3
traindf['TRN_CON_DAE_ACTIVE_MON_01'] = np.where(traindf['TRN_CON_DAE_ACTIVE_MON_01']=='Y', 1, 0)
traindf['BTE_CLOSED_MON_01'] = np.where(traindf['BTE_CLOSED_MON_01']=='Y', 1, 0)
traindf['COC_CLOSED_MON_01'] = np.where(traindf['COC_CLOSED_MON_01']=='Y', 1, 0)
traindf['PL2CC_CLOSED_MON_01'] = np.where(traindf['PL2CC_CLOSED_MON_01']=='Y', 1, 0)
traindf['TRN_CON_DAE_CLOSED_MON_01'] = np.where(traindf['TRN_CON_DAE_CLOSED_MON_01']=='Y', 1, 0)
traindf['LAST_3MTHS_INQ'] = np.where(traindf['LAST_3MTHS_INQ']=='Y', 1, 0)
traindf['PREV_3MTHS_INQ'] = np.where(traindf['PREV_3MTHS_INQ']=='Y', 1, 0)
traindf['CREDIT_ACT'] = np.where(traindf['CREDIT_ACT']=='Y', 1, 0)
traindf['DEBIT_ACT'] = np.where(traindf['DEBIT_ACT']=='Y', 1, 0)
traindf['CEL_TAG'] = np.where(traindf['CEL_TAG']=='Y', 1, 0)
traindf['EL_TAG'] = np.where(traindf['EL_TAG']=='Y', 1, 0)
traindf['AGL_TAG'] = np.where(traindf['AGL_TAG']=='Y', 1, 0)
traindf['CVL_TAG'] = np.where(traindf['CVL_TAG']=='Y', 1, 0)
traindf['AL_TAG'] = np.where(traindf['AL_TAG']=='Y', 1, 0)

In [7]:
# Columns to delete
del traindf['EEG_TAG'],traindf['ZIP_CODE_FINAL'],traindf['TOP_100_CORP'],traindf['TOP_800_CORP'], traindf['EEG_CLOSED']

In [8]:
# Dates
del traindf['MATURITY_GL'],traindf['MATURITY_LAP'],traindf['MATURITY_LAS'],traindf['CLOSED_DATE']

In [9]:
# There are 93 non numerical values
#traindf.EEG_CLOSED.value_counts()

In [10]:
# y
traindf['RESPONDERS'] = np.where(traindf['RESPONDERS']=='Y', 1, 0)

In [11]:
print ('Before One hit encoding', traindf.shape)

Before One hit encoding (300000, 324)


In [12]:
traindf.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,PM_FD_MON_02,PM_FD_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,STMT_CON_DAE_ACTIVE_MON_01,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,STMT_CON_DAE_CLOSED_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,MER_EMI_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,

In [13]:
#del DC_SPEND_MON_03	DC_SPEND_MON_02	DC_SPEND_MON_01	CC_SPEND_MON_04	CC_SPEND_MON_01	CC_SPEND_MON_02	CC_SPEND_MON_03	DC_SPEND_MON_04	PM_FD_MON_01	PM_FD_MON_02	PM_FD_MON_04	CR_LIMIT	STMT_MON_03	STMT_MON_02	STMT_MON_01

In [14]:
traindf = pd.get_dummies(traindf, 
                         columns=['ACT_TYPE','GENDER','LEGAL_ENTITY','OCCUP_ALL_NEW','CC_HOLD','CC_ACTIVE','DC_HOLD',
                                  'DC_ACTIVE','FINANCE_MON_01','JOBS_MON_01','LIFESTYLE_MON_01',
                                  'INMON_01KET_MON_01', 'TRAVEL_MON_01','CHANNEL_CLICK_DISP','DESIGNATION_FINAL',
                                  'TOP14_CITY','TOP9_CITY','PA_PQ_TAG','NEFT_CC_CATEGORY','NEFT_DC_CATEGORY',
                                  'TPT_DC_CATEGORY_MON_01','TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01',
                                  'CARD_AUTOMOBILE_MON_01','CARD_HOTEL_MON_01','CARD_JEWELLERY_MON_01',
                                  'CARD_RESTAURANT_MON_01','CARD_TRAVEL_MON_01','CARD_PRSNL_CARE_MON_01',
                                  'CARD_HOBBY_MON_01','CARD_HOME_DECOR_MON_01','CARD_MEDICAL_MON_01',
                                  'CARD_ENTMNT_MON_01','TOP_CORP_TAG'], drop_first=True)
print ('After One hot encoding',traindf.shape)

After One hot encoding (300000, 568)


In [15]:
traindf.head()

,CUSTOMER_ID,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,CV_TAG,EDU_TAG,TL_TAG,OTHER_LOANS_TAG,LAP_TAG,LAS_TAG,BBG_TAG,HL_TAG,GL_TAG,AL_CLOSED,PL_CLOSED,BL_CLOSED,TWL_CLOSED,CV_CLOSED,LAP_CLOSED,LAS_CLOSED,BBG_CLOSED,GL_CLOSED,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,CC_SPEND_MON_01,CC_SPEND_MON_02,CC_SPEND_MON_03,DC_SPEND_MON_04,PM_FD_MON_01,PM_FD_MON_02,PM_FD_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,LAP_SCRUB_LIVE,LAS_SCRUB_LIVE,PL_SCRUB_LIVE,GL_SCRUB_LIVE,TWL_SCRUB_LIVE,BL_SCRUB_LIVE,CEL_SCRUB_LIVE,TL_SCRUB_LIVE,AL_SCRUB_LIVE,LAP_SCRUB_CLOSED,LAS_SCRUB_CLOSED,PL_SCRUB_CLOSED,GL_SCRUB_CLOSED,TWL_SCRUB_CLOSED,BL_SCRUB_CLOSED,CEL_SCRUB_CLOSED,TL_SCRUB_CLOSED,AL_SCRUB_CLOSED,STMT_CON_DAE_ACTIVE_MON_01,BTE_ACTIVE_MON_01,COC_ACTIVE_MON_01,PL2CC_ACTIVE_MON_01,MER_EMI_ACTIVE_MON_01,TRN_CON_DAE_ACTIVE_MON_01,STMT_CON_DAE_CLOSED_MON_01,BTE_CLOSED_MON_01,COC_CLOSED_MON_01,PL2CC_CLOSED_MON_01,MER_EMI_CLOSED_MON_01,TRN_CON_DAE_CLOSED_MON_01,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CDMAT_MON_01,CDMAT_MON_04,MF_MON_04,MF_MON_01,RD_MON_04,RD_MON_01,FD_MON_04,FD_MON_01,LI_MON_04,LI_MON_01,GI_MON_04,GI_MON_01,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_ENT,DC_SPEND_MON_04_MED,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_HMD,DC_SPEND_MON_04_HBY,DC_SPEND_MON_04_CARE,DC_SPEND_MON_04_TRL,DC_SPEND_MON_04_RST,DC_SPEND_MON_04_JER,DC_SPEND_MON_04_HTL,DC_SPEND_MON_04_ATM,DC_SPEND_MON_03_ENT,DC_SPEND_MON_03_MED,DC_SPEND_MON_03_HMD,DC_SPEND_MON_03_HBY,DC_SPEND_MON_03_CARE,DC_SPEND_MON_03_TRL,DC_SPEND_MON_03_RST,DC_SPEND_MON_03_JER,DC_SPEND_MON_03_HTL,DC_SPEND_MON_03_ATM,DC_SPEND_MON_02_ENT,DC_SPEND_MON_02_MED,DC_SPEND_MON_02_HMD,DC_SPEND_MON_02_HBY,DC_SPEND_MON_02_CARE,DC_SPEND_MON_02_TRL,DC_SPEND_MON_02_RST,DC_SPEND_MON_02_JER,DC_SPEND_MON_02_HTL,DC_SPEND_MON_02_ATM,DC_SPEND_MON_01_ENT,DC_SPEND_MON_01_MED,DC_SPEND_MON_01_HMD,DC_SPEND_MON_01_HBY,DC_SPEND_MON_01_CARE,DC_SPEND_MON_01_TRL,DC_SPEND_MON_01_RST,DC_SPEND_MON_01_JER,DC_SPEND_MON_01_HTL,DC_SPEND_MON_01_ATM,DC_SPEND_MON_05_ENT,DC_SPEND_MON_05_MED,DC_SPEND_MON_05_HMD,DC_SPEND_MON_05_HBY,DC_SPEND_MON_05_CARE,DC_SPEND_MON_05_TRL,DC_SPEND_MON_05_RST,DC_SPEND_MON_05_JER,DC_SPEND_MON_05_HTL,DC_SPEND_MON_05_ATM,CC_SPEND_MON_04_ENT,CC_SPEND_MON_04_MED,CC_SPEND_MON_04_HMD,CC_SPEND_MON_04_HBY,CC_SPEND_MON_04_CARE,CC_SPEND_MON_04_TRL,CC_SPEND_MON_04_RST,CC_SPEND_MON_04_JER,CC_SPEND_MON_04_HTL,CC_SPEND_MON_04_ATM,CC_SPEND_MON_02_ENT,CC_SPEND_MON_02_MED,CC_SPEND_MON_02_HMD,CC_SPEND_MON_02_HBY,CC_SPEND_MON_02_CARE,CC_SPEND_MON_02_TRL,CC_SPEND_MON_02_RST,CC_SPEND_MON_02_JER,CC_SPEND_MON_02_HTL,CC_SPEND_MON_02_ATM,CC_SPEND_MON_01_ENT,CC_SPEND_MON_01_MED,CC_SPEND_MON_01_HMD,CC_SPEND_MON_01_HBY,CC_SPEND_MON_01_CARE,CC_SPEND_MON_01_TRL,CC_SPEND_MON_01_RST,CC_SPEND_MON_01_JER,CC_SPEND_MON_01_HTL,CC_SPEND_MON_01_ATM,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,D_COUNT_L3_MON_01,MAX_C_AMT_L3_MON_01,NEFT_CC_AMOUNT,NEFT_CC_TXN,NEFT_DC_AMOUNT,NEFT_DC_TXN,TPT_DC_AMOUNT_MON_01,TPT_DC_TXN_MON_01,TPT_CC_AMOUNT_MON_01,TPT_CC_TXN_MON_01,IMPS_CC_AMOUNT_MON_01,IMPS_CC_TXN_MON_01,LAS

In [17]:
#testdf = pd.read_csv('/Users/Vishy/Files/AVDatahack/CrossSell/RawData/test.csv')
#testdf = pd.read_csv('/data/test.csv')
#print (testdf.shape)
#testdf.head()

In [18]:
# There are 332 variables in the test data
nonnumcols = testdf.dtypes[testdf.dtypes == "object"].index
print (len(nonnumcols))
#print (nonnumcols)

90


In [19]:
#testdf.BBG_TAG.unique()
# BBG_CLOSED

In [20]:
# replace Binary Nan as flag 0
testdf['BL_TAG'] = np.where(testdf['BL_TAG']=='Y', 1, 0)
testdf['TWL_TAG'] = np.where(testdf['TWL_TAG']=='Y', 1, 0)
testdf['PL_TAG'] = np.where(testdf['PL_TAG']=='Y', 1, 0)
testdf['AL_CNC_TAG'] = np.where(testdf['AL_CNC_TAG']=='Y', 1, 0)
testdf['AGRI_TAG'] = np.where(testdf['AGRI_TAG']=='Y', 1, 0)
testdf['CE_TAG'] = np.where(testdf['CE_TAG']=='Y', 1, 0)
testdf['CV_TAG'] = np.where(testdf['CV_TAG']=='Y', 1, 0)
testdf['EDU_TAG'] = np.where(testdf['EDU_TAG']=='Y', 1, 0)
testdf['TL_TAG'] = np.where(testdf['TL_TAG']=='Y', 1, 0)
testdf['OTHER_LOANS_TAG'] = np.where(testdf['OTHER_LOANS_TAG']=='Y', 1, 0)
testdf['LAP_TAG'] = np.where(testdf['LAP_TAG']=='Y', 1, 0)
testdf['LAS_TAG'] = np.where(testdf['LAS_TAG']=='Y', 1, 0)
testdf['HL_TAG'] = np.where(testdf['HL_TAG']=='Y', 1, 0)
testdf['GL_TAG'] = np.where(testdf['GL_TAG']=='Y', 1, 0)
testdf['AL_CLOSED'] = np.where(testdf['AL_CLOSED']=='Y', 1, 0)
testdf['PL_CLOSED'] = np.where(testdf['PL_CLOSED']=='Y', 1, 0)
testdf['BL_CLOSED'] = np.where(testdf['BL_CLOSED']=='Y', 1, 0)

In [21]:
# replace Binary Nan as flag 0
testdf['TWL_CLOSED'] = np.where(testdf['TWL_CLOSED']=='Y', 1, 0)
testdf['CV_CLOSED'] = np.where(testdf['CV_CLOSED']=='Y', 1, 0)
testdf['LAP_CLOSED'] = np.where(testdf['LAP_CLOSED']=='Y', 1, 0)
testdf['LAS_CLOSED'] = np.where(testdf['LAS_CLOSED']=='Y', 1, 0)
testdf['GL_CLOSED'] = np.where(testdf['GL_CLOSED']=='Y', 1, 0)
testdf['CC_HOLD_MON_03'] = np.where(testdf['CC_HOLD_MON_03']=='Y', 1, 0)
testdf['CC_HOLD_MON_02'] = np.where(testdf['CC_HOLD_MON_02']=='Y', 1, 0)
testdf['CC_HOLD_MON_01'] = np.where(testdf['CC_HOLD_MON_01']=='Y', 1, 0)
testdf['CC_ACTIVE_MON_03'] = np.where(testdf['CC_ACTIVE_MON_03']=='Y', 1, 0)
testdf['CC_ACTIVE_MON_02'] = np.where(testdf['CC_ACTIVE_MON_02']=='Y', 1, 0)
testdf['CC_ACTIVE_MON_01'] = np.where(testdf['CC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['DC_ACTIVE_MON_03'] = np.where(testdf['DC_ACTIVE_MON_03']=='Y', 1, 0)
testdf['DC_ACTIVE_MON_02'] = np.where(testdf['DC_ACTIVE_MON_02']=='Y', 1, 0)
testdf['DC_ACTIVE_MON_01'] = np.where(testdf['DC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['BTE_ACTIVE_MON_01'] = np.where(testdf['BTE_ACTIVE_MON_01']=='Y', 1, 0)
testdf['COC_ACTIVE_MON_01'] = np.where(testdf['COC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['PL2CC_ACTIVE_MON_01'] = np.where(testdf['PL2CC_ACTIVE_MON_01']=='Y', 1, 0)
testdf['MER_EMI_ACTIVE_MON_01'] = np.where(testdf['MER_EMI_ACTIVE_MON_01']=='Y', 1, 0)

In [22]:
# replace Binary Nan as flag 0
testdf['TRN_CON_DAE_ACTIVE_MON_01'] = np.where(testdf['TRN_CON_DAE_ACTIVE_MON_01']=='Y', 1, 0)
testdf['BTE_CLOSED_MON_01'] = np.where(testdf['BTE_CLOSED_MON_01']=='Y', 1, 0)
testdf['COC_CLOSED_MON_01'] = np.where(testdf['COC_CLOSED_MON_01']=='Y', 1, 0)
testdf['PL2CC_CLOSED_MON_01'] = np.where(testdf['PL2CC_CLOSED_MON_01']=='Y', 1, 0)
testdf['TRN_CON_DAE_CLOSED_MON_01'] = np.where(testdf['TRN_CON_DAE_CLOSED_MON_01']=='Y', 1, 0)
testdf['LAST_3MTHS_INQ'] = np.where(testdf['LAST_3MTHS_INQ']=='Y', 1, 0)
testdf['PREV_3MTHS_INQ'] = np.where(testdf['PREV_3MTHS_INQ']=='Y', 1, 0)
testdf['CREDIT_ACT'] = np.where(testdf['CREDIT_ACT']=='Y', 1, 0)
testdf['DEBIT_ACT'] = np.where(testdf['DEBIT_ACT']=='Y', 1, 0)
testdf['CEL_TAG'] = np.where(testdf['CEL_TAG']=='Y', 1, 0)
testdf['EL_TAG'] = np.where(testdf['EL_TAG']=='Y', 1, 0)
testdf['AGL_TAG'] = np.where(testdf['AGL_TAG']=='Y', 1, 0)
testdf['CVL_TAG'] = np.where(testdf['CVL_TAG']=='Y', 1, 0)
testdf['AL_TAG'] = np.where(testdf['AL_TAG']=='Y', 1, 0)

In [23]:
del testdf['BBG_CLOSED'], testdf['BBG_TAG'], traindf['BBG_CLOSED'], traindf['BBG_TAG']

In [24]:
# Columns to delete
del testdf['EEG_TAG'],testdf['ZIP_CODE_FINAL'],testdf['TOP_100_CORP'],testdf['TOP_800_CORP'], testdf['EEG_CLOSED']

In [25]:
# Dates
del testdf['MATURITY_GL'],testdf['MATURITY_LAP'],testdf['MATURITY_LAS'],testdf['CLOSED_DATE']

In [26]:
print ('Before One hit encoding', testdf.shape)

Before One hit encoding (200000, 321)


In [27]:
testdf = pd.get_dummies(testdf, 
                            columns=['ACT_TYPE','GENDER','LEGAL_ENTITY','OCCUP_ALL_NEW','CC_HOLD','CC_ACTIVE','DC_HOLD',
                                  'DC_ACTIVE','FINANCE_MON_01','JOBS_MON_01','LIFESTYLE_MON_01',
                                  'INMON_01KET_MON_01', 'TRAVEL_MON_01','CHANNEL_CLICK_DISP','DESIGNATION_FINAL',
                                  'TOP14_CITY','TOP9_CITY','PA_PQ_TAG','NEFT_CC_CATEGORY','NEFT_DC_CATEGORY',
                                  'TPT_DC_CATEGORY_MON_01','TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01',
                                  'CARD_AUTOMOBILE_MON_01','CARD_HOTEL_MON_01','CARD_JEWELLERY_MON_01',
                                  'CARD_RESTAURANT_MON_01','CARD_TRAVEL_MON_01','CARD_PRSNL_CARE_MON_01',
                                  'CARD_HOBBY_MON_01','CARD_HOME_DECOR_MON_01','CARD_MEDICAL_MON_01',
                                  'CARD_ENTMNT_MON_01','TOP_CORP_TAG'], drop_first=True)
print ('After One hot encoding',testdf.shape)

After One hot encoding (200000, 550)


In [28]:
print (traindf.shape, testdf.shape)

(300000, 566) (200000, 550)


In [29]:
listA = list(traindf.columns.values)
listB = list(testdf.columns.values)

In [30]:
print (len(listA), len(listB))

566 550


In [31]:
"""
for item in listA:
    if item not in listB:
        print (item)
"""

'\nfor item in listA:\n    if item not in listB:\n        print (item)\n'

In [32]:
testdf['LEGAL_ENTITY_FIRM'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_103K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_107K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_112K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_115K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_121K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_122K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_1250K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_129K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_134K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_182K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_183K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_192K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_2000K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_205K'] = 0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_233K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_241K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_246K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_247K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_258K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_292K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_333K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_400K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_476K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_583K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_667K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_700K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_833K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_94K'],testdf['PA_PQ_TAG_PQ_WITH_INCOME_97K']=0,0,0
testdf['PA_PQ_TAG_PQ_WITH_INCOME_98K'],testdf['NEFT_CC_CATEGORY_WEDDING'],testdf['NEFT_DC_CATEGORY_RETURN']=0,0,0
testdf['TPT_DC_CATEGORY_MON_01_OPERATION'],testdf['IMPS_CC_CATEGORY_MON_01_JEWELLERY']=0,0

In [33]:
"""
for item in listB:
    if item not in listA:
        print (item)
"""

'\nfor item in listB:\n    if item not in listA:\n        print (item)\n'

In [18]:
traindf['LEGAL_ENTITY_PROPRIETERSHIP'],traindf['LEGAL_ENTITY_TRUSTS'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_118K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_124K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_130K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_136K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_143K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_144K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_158K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_185K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_188K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_196K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_260K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_317K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_3333K']=0,0,0
traindf['PA_PQ_TAG_PQ_WITH_INCOME_407K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_4167K'],traindf['PA_PQ_TAG_PQ_WITH_INCOME_600K']=0,0,0
traindf['NEFT_CC_CATEGORY_CC_PAYMENT'],traindf['NEFT_DC_CATEGORY_BUILDER']=0,0

In [26]:
print (traindf.shape)#, testdf.shape)

(300000, 588)


In [24]:
allcols = list(traindf.columns.values)
print (len(allcols))

588


In [28]:
df = pd.DataFrame({'col':allcols})

In [30]:
df.to_csv('cols.csv')

In [37]:
testdf['RESPONDERS']=0

In [38]:
traindf = traindf[allcols]
testdf = testdf[allcols]
del testdf['RESPONDERS']

In [21]:
target_train = traindf['RESPONDERS'].values
#id_test = testdf['CUSTOMER_ID'].values

train = np.array(traindf.drop(['RESPONDERS','CUSTOMER_ID'], axis = 1))
#test = np.array(testdf.drop(['CUSTOMER_ID'], axis = 1))

xgb_preds = []

In [22]:
train.shape

(300000, 586)

In [ ]:
feats = pd.read(csv)

In [40]:
traindf.RESPONDERS.value_counts()

0    295388
1      4612
Name: RESPONDERS, dtype: int64

In [41]:
K = 3
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [42]:
start = time()

for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.5, 'colsample_bytree': 0.9, 
                  'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, #feval=lift_xgb,
                      verbose_eval=100, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time()

[0]	train-auc:0.81965	valid-auc:0.821147
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.867628	valid-auc:0.862438
[200]	train-auc:0.884469	valid-auc:0.879805
[300]	train-auc:0.896068	valid-auc:0.886808
[400]	train-auc:0.905492	valid-auc:0.890203
[500]	train-auc:0.912406	valid-auc:0.891865
[600]	train-auc:0.918144	valid-auc:0.893335
[700]	train-auc:0.922682	valid-auc:0.893924
[800]	train-auc:0.927059	valid-auc:0.894511
[900]	train-auc:0.930747	valid-auc:0.894776
[1000]	train-auc:0.934342	valid-auc:0.894839
[1100]	train-auc:0.93761	valid-auc:0.895105
Stopping. Best iteration:
[1140]	train-auc:0.938883	valid-auc:0.895263

[0]	train-auc:0.817165	valid-auc:0.808838
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.872285	valid-auc:0.85888
[200]	train-auc:0.88893	valid-a

NameError: name 'k' is not defined

In [43]:
print ('Total Time taken for ',K,' cross validation is', end-start)

Total Time taken for  3  cross validation is 4413.487955570221


In [44]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out = pd.DataFrame({'CUSTOMER_ID': id_test, 'RESPONDERS': preds})

In [43]:
#out.to_csv("pred1.csv", index=False)

In [49]:
# Get Feature Importance from the classifier
import matplotlib.pyplot as plt
feature_importance = model.get_fscore()
# Normalize The Features
#feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(16, 20))
plt.barh(pos, feature_importance[sorted_idx], align='center', color='#7A68A6')
#plt.yticks(pos, np.asanyarray(NTraindf.columns.tolist())[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

TypeError: unhashable type: 'numpy.ndarray'

In [53]:
feats = pd.DataFrame.from_dict(feature_importance, orient='index')

In [56]:
feats.to_csv('feats.csv')

In [ ]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(NTraindf.columns, clf1.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance', ascending=False)
importances = importances.iloc[:, :]
importances = importances.T
importances.to_csv('importances.csv')
importances